In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#url to be scraped
news_url = 'https://redplanetscience.com/'
feature_url = 'https://spaceimages-mars.com/'
facts_url = 'https://galaxyfacts-mars.com/'
astro_geo_url = 'https://marshemispheres.com/'

In [3]:
page = requests.get(news_url)

In [4]:
soup = bs(page.text, 'html.parser')

In [5]:
print(soup)

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
<link href="css/font.css" rel="stylesheet" type="text/css"/>
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet">
<title>News - Mars Exploration Program</title>
</link></head>
<body>
<div class="col-md-12">
<div class="row">
<nav class="navbar navbar-expand-lg navbar-light fixed-top">
<div class="container-fluid">
<a class="navbar-brand" href="#">
<img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
<span class="logo1">

In [6]:
news_title = soup.find_all('div', class_="content_title")
news_p = soup.find_all('div', class_='article_teaser_body')

In [7]:
for i in news_title:
    print(i)


In [ ]:
for i in news_p:
    print(i)

In [8]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Get LATEST driver version for 90.0.4430
Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\fernb\.wdm\drivers\chromedriver\win32\90.0.4430.24]


In [9]:
browser.visit(news_url)

In [10]:
html = browser.html
soup = bs(html, 'lxml')

In [11]:
print(soup)

<html><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
<link href="css/font.css" rel="stylesheet" type="text/css"/>
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
<title>News - Mars Exploration Program</title>
</head>
<body>
<div class="col-md-12">
<div class="row">
<nav class="navbar navbar-expand-lg navbar-light fixed-top">
<div class="container-fluid">
<a class="navbar-brand" href="#">
<img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
<span class="logo1">Exploration Program</spa

In [12]:
news_title = soup.find_all('div', class_="content_title")[0].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text

In [13]:
print(news_title)
print(news_p)


NASA Adds Return Sample Scientists to Mars 2020 Leadership Team
The leadership council for Mars 2020 science added two new members who represent the interests of scientists destined to handle and study the first samples from Red Planet.


In [14]:
browser.visit(feature_url)

In [15]:
links_found = browser.links.find_by_partial_href('feature').first['href']
print(links_found)

https://spaceimages-mars.com/image/featured/mars1.jpg


AttributeError: 'str' object has no attribute 'value'

In [65]:
browser.quit()

In [43]:
tables = pd.read_html(facts_url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [44]:
df = tables[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [69]:
browser.visit(astro_geo_url)

In [70]:



image_links = browser.find_by_css('.product-item').links.find_by_partial_text('Enhance')

listoflinks = []
for i in range(4):
    listoflinks.append(image_links[i]['href'])
        


        
    


In [91]:
tifflist = {}
for i in listoflinks:
    browser.visit(i)
    tiff = browser.links.find_by_text('Original')
    name = browser.find_by_css('.title').first.value.rstrip('Enhanced').strip()
    tifflist[name] = tiff['href']
    

    

In [92]:
print(tifflist)
print(listoflinks)

{'Cerberus Hemisphere': 'https://marshemispheres.com/images/cerberus_enhanced.tif', 'Schiaparelli Hemisphere': 'https://marshemispheres.com/images/schiaparelli_enhanced.tif', 'Syrtis Major Hemisphere': 'https://marshemispheres.com/images/syrtis_major_enhanced.tif', 'Valles Marineris Hemisphere': 'https://marshemispheres.com/images/valles_marineris_enhanced.tif'}
['https://marshemispheres.com/cerberus.html', 'https://marshemispheres.com/schiaparelli.html', 'https://marshemispheres.com/syrtis.html', 'https://marshemispheres.com/valles.html']
